In [1]:
from coral_imports import *

# Read in Pipeline and Libraries

In [2]:
base = os.path.join(os.getcwd(), "analysis", "configs", "base.yaml")
base_float = os.path.join(os.getcwd(), "analysis", "configs", "base_float.yaml")
library = os.path.join(os.getcwd(), "analysis", "library")
base_baseline = os.path.join(os.getcwd(), "analysis", "pipelines", "base_baseline_pipeline.csv")
pipeline = Pipeline(base_baseline, base, base_float, enforce_feeders=True)

# Create DFs based on CORAL runs

### Baseline, Base

### Baseline, unconstrained vessels

### Baseline, unconstrained ports

# Plot Comparison of delays